In [1]:
import os
import sys
import cv2
import time
import argparse
import numpy as np

In [2]:
CLASSES = "cartoon.names"
CONFIG = "yolov2.cfg"
WEIGHTS = "cartoon_yolo.weights"
FILE = "test.jpg"
CONFIDENCE = 0.5
NMS_THRESHOLD = 0.3
SCALE = 0.00392
MODE = "image" 

In [3]:
classes = None
try:
    with open(CLASSES, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
        f.close()
    print("[INFO] Classes loaded successfully")
except:
    sys.exit("[ERROR] File Exception Occured")
    
COLORS = [
    [255, 0, 0],
    [0, 255, 0],
    [0, 0, 255],
    [255, 255, 0],
    [0, 255, 255]
]

[INFO] Classes loaded successfully


In [5]:
net = cv2.dnn.readNet(WEIGHTS, CONFIG)
layers_names = net.getLayerNames()
output_layers = [layers_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print("[INFO] Model Loaded")

[INFO] Model Loaded


In [6]:
def draw_BBox(image, class_id, confidence, x, y, x_plus_width, y_plus_height):
    label = str(classes[class_id]) + " " +str(round(confidence * 100, 2)) + "%"
    color = COLORS[class_id]
    cv2.rectangle(image, (x, y), (x_plus_width, y_plus_height), color, 2)
    cv2.putText(image, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

In [9]:
def process_Image(image, index):

    print("[INFO] Processing Frame : ", (index+1))

    #Retriving dimensions of image
    print("[INFO] Image Dimension : ", image.shape)
    width = image.shape[1]
    height = image.shape[0]

    #Create Input blob, and set input for network
    blob = cv2.dnn.blobFromImage(image, SCALE, (416, 416), (0, 0, 0), True, crop = False)
    net.setInput(blob)

    #Forward Propogation - inference
    outs = net.forward(output_layers)

    #Initialization of variables (lists)
    class_ids = []
    confidences = []
    boxes = []

    #For each detection from each output layer, get the confidence, class id and BBox params
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            #Thresholding
            if confidence > CONFIDENCE:
                #Calculating BBox params
                c_x = int(detection[0] * width)
                c_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = c_x - w / 2
                y = c_y - h / 2

                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    #Applying NMS
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE, NMS_THRESHOLD)

    #Draw final BBoxes
    for ind in indices:
        i = ind[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_BBox(image, class_ids[i], confidences[i], round(x), round(y), round(x + w), round(y + h))
        print("[Prediction] Class : ", classes[class_ids[i]])
        print("[Prediction] Score : ", round(confidences[i] * 100, 6))
    
    #Storing Image
    if MODE == "image":
        cv2.imwrite("output.jpg", image)

In [10]:
start = time.time()
#Loading input file and preocessing
if MODE == "image":
    img = cv2.imread(FILE, cv2.IMREAD_COLOR)
    process_Image(img, 1)
end = time.time()
total_time = round(end-start,2)
print("[INFO] Time : {} sec" .format(total_time))

[INFO] Processing Frame :  2
[INFO] Image Dimension :  (507, 498, 3)
[Prediction] Class :  shinchan
[Prediction] Score :  91.417855
[INFO] Time : 0.24 sec
